In [10]:
# import pickle to save the model and processed data
import pickle as pkl

from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVC

from sklearn.metrics import classification_report


import pandas as pd
import numpy as np

In [3]:
#data = pd.read_csv('DataSets/Processed_Data.csv',delimiter = ',')
data = pd.read_csv('Processed_Data.csv',delimiter = ',')

In [4]:
data.dropna(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 864112 entries, 0 to 864375
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   generated           864112 non-null  int64  
 1   text                864112 non-null  object 
 2   avg_len_sentences   864112 non-null  float64
 3   words_count         864112 non-null  int64  
 4   punctuations_count  864112 non-null  int64  
 5   readability_score   864112 non-null  float64
 6   stop_word_ratio     864112 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 52.7+ MB


In [5]:
'''null_rows = data.isnull().sum(axis=1)

# Filter for rows with at least one null value
rows_with_nulls = data[null_rows > 0]

rows_with_nulls.info()'''

'null_rows = data.isnull().sum(axis=1)\n\n# Filter for rows with at least one null value\nrows_with_nulls = data[null_rows > 0]\n\nrows_with_nulls.info()'

In [6]:
X=data[['text','avg_len_sentences','punctuations_count','words_count','stop_word_ratio','readability_score']]
y=data['generated']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
text_pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer()),  # Step 1: CountVectorizer
    ('tfidf_transformer', TfidfTransformer())  # Step 2: TF-IDF Transformation
])

numerical_pipeline = Pipeline([
    ('scaler', MinMaxScaler())  # Scale numerical features
])

# Combine text and numerical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('text', text_pipeline, 'text'),  # Apply text pipeline to 'text' column
    ('numerical', numerical_pipeline, ['avg_len_sentences', 'punctuations_count','readability_score','words_count','stop_word_ratio'])  # Apply numerical pipeline
])

# Final pipeline with preprocessor and model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())  # Replace with any model
])

In [9]:
'''pipeline.fit(X_train, y_train)'''

'pipeline.fit(X_train, y_train)'

In [10]:
'''y_pred = pipeline.predict(X_test)'''

'y_pred = pipeline.predict(X_test)'

In [11]:
'''print(classification_report(y_test,y_pred))'''

'print(classification_report(y_test,y_pred))'

In [12]:
'''with open('NaiveBayesClassifier.pkl', 'wb') as file:
    pkl.dump(pipeline, file)'''

"with open('NaiveBayesClassifier.pkl', 'wb') as file:\n    pkl.dump(pipeline, file)"

In [13]:
'''pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())  # Replace with any model
])'''

"pipeline2 = Pipeline([\n    ('preprocessor', preprocessor),\n    ('classifier', LogisticRegression())  # Replace with any model\n])"

In [14]:
'''pipeline2.fit(X_train, y_train)'''

'pipeline2.fit(X_train, y_train)'

In [15]:
'''y_pred= pipeline2.predict(X_test)'''

'y_pred= pipeline2.predict(X_test)'

In [16]:
'''print(classification_report(y_test,y_pred))'''

'print(classification_report(y_test,y_pred))'

In [17]:
'''with open('LogisticRegressionClassifier.pkl', 'wb') as file:
    pkl.dump(pipeline2, file)'''

"with open('LogisticRegressionClassifier.pkl', 'wb') as file:\n    pkl.dump(pipeline2, file)"

In [13]:
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', DecisionTreeClassifier())  # Replace with any model
])

In [19]:
'''param_grid = {
    "clf__n_estimators": [100, 500, 1000],
    "clf__max_depth": [1, 5, 10, 25],
    "clf__max_features": [*np.arange(0.1, 1.1, 0.1)],
}'''

'param_grid = {\n    "clf__n_estimators": [100, 500, 1000],\n    "clf__max_depth": [1, 5, 10, 25],\n    "clf__max_features": [*np.arange(0.1, 1.1, 0.1)],\n}'

In [20]:
'''gs = GridSearchCV(pipeline3, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1000)
gs.fit(X_train, y_train)
print(gs.score())'''

'gs = GridSearchCV(pipeline3, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1000)\ngs.fit(X_train, y_train)\nprint(gs.score())'

In [14]:
pipeline3.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('count_vectorizer',
                                                                   CountVectorizer()),
                                                                  ('tfidf_transformer',
                                                                   TfidfTransformer())]),
                                                  'text'),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['avg_len_sentences',
                                                   'punctuations_count',
                                                   'readability_score',
                                                   'words_count',
                                                   'stop_word_ratio'])])),
                ('clf', DecisionTreeClassifier())])

In [15]:
y_pred = pipeline3.predict(X_test)

In [16]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84    149833
           1       0.79      0.79      0.79    109401

    accuracy                           0.82    259234
   macro avg       0.82      0.82      0.82    259234
weighted avg       0.82      0.82      0.82    259234



In [17]:
with open('DecisionTreeClassifier.pkl', 'wb') as file:
    pkl.dump(pipeline3, file)